## Welcome!
Welcome to this Colab notebook! We will walk you through several examples of exploring similar and dissimilar counties in US based on statistical variable values in Data Commons datasets. There are primarily two types of data we define here:
1. Snapshot. We look at the statistical variable value at a given time (most likely the latest observation). The similarity criterion we used is absolute difference between two values.
2. Time series. We look at all available statistical variable values as a time series. The similarity criterion can be either Pearson correlation (pearson), or Mean Square Error (mse).

We hope the examples can help you better understand Data Commons and how to use it, so that further research / analysis can be conducted easily

## Load Packages

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
from scipy.stats import rankdata, pearsonr
import datacommons as dc
import numpy as np
import copy
import pandas as pd
from urllib.request import urlopen
import json
import plotly.express as px


## Argument Parsing
1. query_dcid: this is the county we are looking at.
2. restrict_same_state: True if we want to only compare query county with counties in the same state.
3. K: number of similar/dissimilar counties to retrieve.

In [ ]:
query_dcid = 'geoId/06085' # Santa Clara county
# query_dcid = 'geoId/15001' # Hawaii county
restrict_same_state = True # only retrieving and ranking places that are in the same state
K = 10

Get all the state and county information we need

In [ ]:
if restrict_same_state:
    place_domain = dc.get_property_values([query_dcid], 'containedInPlace')[query_dcid][0]
else:
    place_domain = 'country/USA'
all_states = dc.get_places_in(['country/USA'], "State")['country/USA']
all_states = dc.get_property_values(all_states, "name")
all_counties = dc.get_places_in([place_domain], "County")[place_domain]
all_counties = dc.get_property_values(all_counties, "name")
id_list = sorted(list(all_counties.keys()))

## Single statistical variable snapshot case
First, let's look at one statistical variable say unemployment rate for snapshot case. For this statistical variable, it does not make sense to look at it on a per capita basis, so we set it to False

In [ ]:
statvars = ['UnemploymentRate_Person']
per_capita = [False]
time_series = [False]
time_series_similarity_types = ['pearson']  # either pearson or mse for time series similarity metric
statvar_display_names = ['Unemployment Rate']

## Get all the raw data we need

In [ ]:
def get_data_and_population(all_counties, statvars):
    data = []
    for i in range(len(statvars)):
        data.append(dc.get_stats(all_counties, statvars[i], obs_dates="all"))
    population = dc.get_stats(all_counties, 'Count_Person', obs_dates='all')
    return data, population

In [ ]:
data, population = get_data_and_population(all_counties, statvars)

## Normalize the data if applicable and compute distance
Here we will first normalize the data if per_capita is True
We then for each statistical variable, compute distance between query county and every other county. The smaller the distance is, the more similar two counties are based on this statistical variable. 

In [ ]:
def normalize_data(data, population, per_capita):
    data = copy.deepcopy(data)
    # if cannot normalize all values for a place, drop this place
    assert len(data) == len(per_capita)
    for i in range(len(per_capita)):
        if per_capita[i]:
            current_data = data[i]
            for k in current_data.keys():
                if current_data[k] is not None and population[k] is not None:
                    current_data_place = current_data[k]['data']
                    population_place = population[k]['data']
                    data[i][k]['data'] = normalize_single(current_data_place, population_place)
    return data

def normalize_single(data_dict, population_dict):
    new_data_dict = {}
    population_years = sorted(list(population_dict.keys()))
    for k in data_dict.keys():
        if k[:4] in population_years:
            new_data_dict[k] = data_dict[k] / population_dict[k[:4]]
        elif int(k[:4]) > int(population_years[-1]):
            new_data_dict[k] = data_dict[k] / population_dict[population_years[-1]]
    return new_data_dict

In [ ]:
def compute_distances(query_dcid, data, statvars, time_series, time_series_similarity_types):
    distance_lists = []
    for i in range(len(data)):
        print('processing {0} data'.format(statvars[i]))
        distance_lists.append(get_distance_all(data[i], query_dcid, time_series[i], time_series_similarity_types[i]))
    return distance_lists

def get_distance_all(data_dict, query, time_series, time_series_similarity_type):
    id_list = []
    distance_list = []
    if 'data' not in data_dict[query]:
        return {}
    query_dict = data_dict[query]['data']
    for k in data_dict.keys():
        id_list.append(k)
        # if 'data' in data_dict[k].keys():  # Before API change
        if data_dict[k] is not None and 'data' in data_dict[k].keys():
            distance_list.append(get_distance_pair(query_dict, data_dict[k]['data'], time_series, time_series_similarity_type))
        else:
            distance_list.append(get_distance_pair(query_dict, {}, time_series, time_series_similarity_type))

    id_list, distance_list = (list(t) for t in zip(*sorted(zip(id_list, distance_list))))
    return distance_list

def get_distance_pair(dict_a, dict_b, time_series, time_series_similarity_type):
    common_dates = get_common_keys(dict_a, dict_b)
    if len(common_dates) == 0:
        return float('inf')
    common_series_a = [dict_a[date] for date in common_dates]
    common_series_b = [dict_b[date] for date in common_dates]
    if time_series:
        if time_series_similarity_type == 'pearson':
            if len(common_series_a) < 2:
                return float('inf')  # due to insufficient number of data points
            else:
                r, p = pearsonr(common_series_a, common_series_b)  # need to convert correlation to some distance measure
                return 1 - r
        elif time_series_similarity_type == 'mse':
            return np.square(np.subtract(common_series_a, common_series_b)).mean()
        else:
            raise NotImplementedError
    else:
        return abs(common_series_a[-1] - common_series_b[-1])

def get_common_keys(dict_a, dict_b):
    return sorted(list(set(dict_a.keys()).intersection(set(dict_b.keys()))))

def convert_dict_to_list(d):
    sorted_dict = sorted(d.items())  # sorted by key, return a list of tuples
    x, y = zip(*sorted_dict)  # unpack a list of pairs into two tuples
    return x, y

In [ ]:
normalized_data = normalize_data(data, population, per_capita)
distance_lists = compute_distances(query_dcid, data, statvars, time_series, time_series_similarity_types)

## Retrieving similar counties
First we will retrieve counties that are similar to Santa Clara.
This is achieved by setting similar to True

In [ ]:
def rank_all_remove_inf(id_list, distance_lists, similar):
    distances = np.array(distance_lists)
    inf_indices = list(set(np.where(distances == float('inf'))[1]))
    new_id_list = np.delete(id_list, inf_indices)
    new_distances = np.delete(distances, inf_indices, axis=1)
    assert len(new_id_list) == new_distances.shape[1]
    ranks = np.zeros((new_distances.shape[1]))
    for i in range(new_distances.shape[0]):
        ranks = ranks + rankdata(new_distances[i, :], method='min')
    if not similar:
        topK = np.argsort(-ranks)
    else:
        topK = np.argsort(ranks)
    return new_id_list[topK]

In [ ]:
similar = True # True for retrieving least similar entities
all_dcid_ranked = rank_all_remove_inf(id_list, distance_lists, similar)
# delete query dcid and add back to the front approriately due to tie conditions
all_dcid_ranked = all_dcid_ranked[all_dcid_ranked != query_dcid]
all_dcid_ranked = np.insert(all_dcid_ranked, 0, query_dcid)

## Visualize our results
For time series data, we show the retrieval counties by plotting them pairwise per graph.
For snapshot data, we show the retrieval counties in a table
Let's Define some plotting functions

In [ ]:
def multiple_plot(x_list, y_list, x_label, y_label, legends):
    assert len(x_list) == len(y_list)
    assert len(x_list) == len(legends)
    for i in range(len(x_list)):
        x = x_list[i]
        y = y_list[i]
        x = matplotlib.dates.datestr2num(x)
        plt.plot_date(x, y, label=legends[i])
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend(loc="best")
    plt.show()
def visualize_all(normalized_data, query_dcid, all_dcid_ranked, all_counties, statvar_display_names, k, time_series, all_states):
    snapshot_indices = []
    for i in range(len(time_series)):
        if not time_series[i]:
            snapshot_indices.append(i)
    df = visualize_snapshots(normalized_data, query_dcid, all_dcid_ranked, all_counties, statvar_display_names, snapshot_indices, all_states)
    for i in range(len(time_series)):
        if time_series[i]:
            print('===================================Plotting {0} time series==================================='.format(statvar_display_names[i]))
            visualize_time_series_single(normalized_data[i], query_dcid, all_dcid_ranked, all_counties, statvar_display_names[i], k)
    return df


def visualize_time_series_single(current_data, query_dcid, all_dcid_ranked, all_counties, statvar_display_name, k):
    query_x, query_y = convert_dict_to_list(current_data[query_dcid]['data'])
    for j in range(k):
        current_x, current_y = convert_dict_to_list(current_data[all_dcid_ranked[j + 1]]['data'])
        multiple_plot([query_x, current_x], [query_y, current_y], 'time', statvar_display_name, [all_counties[query_dcid][0], all_counties[all_dcid_ranked[j + 1]][0]])


def visualize_snapshots(normalized_data, query_dcid, all_dcid_ranked, all_counties, statvar_display_names, snapshot_indices, all_states):
    results = []
    query_series_list = []
    for index in snapshot_indices:
        query_series_list.append(normalized_data[index][query_dcid]['data'])
    for i in range(len(all_dcid_ranked)):
        dcid = all_dcid_ranked[i]
        current = [dcid, all_counties[dcid][0], all_states[dcid[:8]][0]]
        for j in range(len(snapshot_indices)):
            current_series = normalized_data[snapshot_indices[j]][dcid]['data']
            date = get_common_keys(query_series_list[j], current_series)[-1]
            value = current_series[date]
            current.extend([value, date])
        results.append(current)
    columns = ['dcid', 'name', 'state']
    for i in range(len(snapshot_indices)):
        columns.extend([statvar_display_names[snapshot_indices[i]], statvar_display_names[snapshot_indices[i]] + '_date'])
    df = pd.DataFrame(results, columns=columns)
    return df

In [ ]:
df = visualize_all(normalized_data, query_dcid, all_dcid_ranked, all_counties, statvar_display_names, K, time_series, all_states)
df.head(K + 1)

## Now let's retrieve dissimilar counties

In [ ]:
similar = False # True for retrieving least similar entities
all_dcid_ranked = rank_all_remove_inf(id_list, distance_lists, similar)
# delete query dcid and add back to the front approriately due to tie conditions
all_dcid_ranked = all_dcid_ranked[all_dcid_ranked != query_dcid]
all_dcid_ranked = np.insert(all_dcid_ranked, 0, query_dcid)

In [ ]:
df = visualize_all(normalized_data, query_dcid, all_dcid_ranked, all_counties, statvar_display_names, K, time_series, all_states)
df.head(K + 1)

## Encapsulate the above functions to one single function
After we understand the functionality of each function, let's define one function that gets the data, normalize it, compute distance and display results

In [ ]:
def retrieve_compute_visualize(query_dcid, restrict_same_state, statvars, statvar_display_names, per_capita, time_series, time_series_similarity_types, similar, k):
    if restrict_same_state:
        place_domain = dc.get_property_values([query_dcid], 'containedInPlace')[query_dcid][0]
    else:
        place_domain = 'country/USA'
    all_states = dc.get_places_in(['country/USA'], "State")['country/USA']
    all_states = dc.get_property_values(all_states, "name")
    all_counties = dc.get_places_in([place_domain], "County")[place_domain]
    all_counties = dc.get_property_values(all_counties, "name")
    id_list = sorted(list(all_counties.keys()))
    data, population = get_data_and_population(all_counties, statvars)
    normalized_data = normalize_data(data, population, per_capita)
    distance_lists = compute_distances(query_dcid, data, statvars, time_series, time_series_similarity_types)
    all_dcid_ranked = rank_all_remove_inf(id_list, distance_lists, similar)
    all_dcid_ranked = all_dcid_ranked[all_dcid_ranked != query_dcid]
    all_dcid_ranked = np.insert(all_dcid_ranked, 0, query_dcid)
    df = visualize_all(normalized_data, query_dcid, all_dcid_ranked, all_counties, statvar_display_names, k, time_series, all_states)
    return df

## Single statistical variable time series case
We again look at Santa Clara county but this time focus on median income in time series. We choose Pearson correlation to measure similarity and once again, restrict ourselves to California state only

In [ ]:
query_dcid = 'geoId/06085' # Santa Clara county
restrict_same_state = True
K = 10
statvars = ['Median_Income_Person']
per_capita = [False]
time_series = [True]
time_series_similarity_types = ['pearson']
statvar_display_names = ['Median Income']
similar = True

We observe indeed the median income time series plots show similar trend on median income even though their difference is consistently large in some cases. This is expected since we used Pearson correlation coefficient, which unlike MSE, is looking at trend rather than absolute value differences.

In [ ]:
df = retrieve_compute_visualize(query_dcid, restrict_same_state, statvars, statvar_display_names, per_capita, time_series, time_series_similarity_types, similar, K)
df.head(K + 1)

## Change it to mse and rerun everything
Now every corresponding observations are much more closer

In [ ]:
time_series_similarity_types = ['mse']
df = retrieve_compute_visualize(query_dcid, restrict_same_state, statvars, statvar_display_names, per_capita, time_series, time_series_similarity_types, similar, K)
df.head(K + 1)

## We can even look at both metrics at the same time
In this case, for each metric, the algorithm considers it indepedently and assign ranks to every places based on the comuputed similarity. The rank is then aggregated to produce the final retrieval results.

You will see two identical sets of figures. This is due to the fact that our algorithm actually treats the two metrics as two different statistical variables, and it will plot for the retrieval results for each statistical variables indepdently. This is helpful when we are looking at multiple statistical variables. 

In [ ]:
statvars = ['Median_Income_Person', 'Median_Income_Person']
per_capita = [False, False]
time_series = [True, True]
time_series_similarity_types = ['mse', 'pearson']
statvar_display_names = ['Median Income Pearson', 'Median Income MSE']
df = retrieve_compute_visualize(query_dcid, restrict_same_state, statvars, statvar_display_names, per_capita, time_series, time_series_similarity_types, similar, K)
df.head(K + 1)

## Now we can look at multiple statistical variables at the same time!
We look at three statistical variables: unemployment rate, median income, labour force participation rate. Since Data Commons does not have per capita statistics for labour force, we set per capita to True for that variable. We look at the first two statistical variables with time series based on Pearson correlation and labour force participation rate as snapshot. The plots for unemployment rate and median income show very similar trend between Santa Clara county and retrieved places. The labour force participation rate is very close between query county and retrieved counties as well.
This time, we will look at all counties in US, not just California. So it may take a while to get the data.

In [ ]:
statvars = ['UnemploymentRate_Person', 'Median_Income_Person', 'Count_Person_InLaborForce']
per_capita = [False, False, True]
time_series = [True, True, False]
restrict_same_state = False  # look at all counties in US
statvar_display_names = ['Unemployment Rate', 'Median Income', 'Labor Force Per Capita']
time_series_similarity_types = ['pearson', 'pearson', '']  # omit the last element since we are looking at snapshot case for labour force
df = retrieve_compute_visualize(query_dcid, restrict_same_state, statvars, statvar_display_names, per_capita, time_series, time_series_similarity_types, similar, K)
df.head(K + 1)

## Choropleth Visualization

Since we just computed similarity for all counties, we will display a choropleth (heatmap) allowing us to visualize a geographical distribution of counties that are similar to Santa Clara based on the statistical variables and settings we just described. In the below heatmap, the darker the color is, the similar the county is with Santa Clara. This may take a while.

In [ ]:
def heatmap_visualization(df, display_topK, k):
    filename = 'us_counties.json'
    with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
        counties_geo = json.load(response)
    df_choro = copy.deepcopy(df)
    df_choro['dcid'] = df_choro['dcid'].apply(lambda x: str(x).replace('geoId/', ''))
    if display_topK:
        df_choro = df_choro[:k + 1]
    fig = px.choropleth(df_choro, geojson=counties_geo, locations='dcid', color=df_choro.index,
                        color_continuous_scale="Viridis",
                        range_color=(0, len(df_choro) - 1),
                        scope="usa",
                        labels={'index': 'rank'}, hover_data={'name'}
                        )
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.show()

In [ ]:
display_topK = False
heatmap_visualization(df, display_topK, K)

## What's Next?
You can explore more statistical variable combinations with different settings (snapshot/time series, per capita or different similarity metrics or all of them)! Also don't forget you can change your query county to somewhere else. We provide a few suggestions below just to get you started! Good luck with exploring!

In [ ]:
query_dcid = 'geoId/36103' # Suffolk county
restrict_same_state = False
statvars = ['Amount_EconomicActivity_GrossDomesticProduction_RealValue', 'Count_Establishment', 'WagesAnnual_Establishment']
statvar_display_names = ['GDP', 'Number of Establishments', 'Annual Wages of Establishments']
per_capita = [True, True, True]
time_series = [True, True, False]
time_series_similarity_types = ['pearson', 'pearson', '']
similar = True
K = 10
df = retrieve_compute_visualize(query_dcid, restrict_same_state, statvars, statvar_display_names, per_capita, time_series, time_series_similarity_types, similar, K)
df.head(K + 1)

In [ ]:
query_dcid = 'geoId/15001' # Hawaii county
restrict_same_state = False
statvars = ['CumulativeCount_MedicalConditionIncident_COVID_19_ConfirmedOrProbableCase', 'CumulativeCount_MedicalConditionIncident_COVID_19_PatientDeceased']
statvar_display_names = ['Confirmed or Probable Cases', 'Patients Deceased']
per_capita = [False, False]
time_series = [False, False]
time_series_similarity_types = ['', '']
similar = True
K = 10
df = retrieve_compute_visualize(query_dcid, restrict_same_state, statvars, statvar_display_names, per_capita, time_series, time_series_similarity_types, similar, K)
df.head(K + 1)